In [ ]:
#https://discuss.huggingface.co/t/finetuning-bart-using-custom-loss/4060/6

## LOADING ALL DATASET

In [1]:
import pandas as pd
df_mcqn = pd.read_csv("mcq_neighbours.csv")
df_mcqs = pd.read_csv("mcq_street.csv")
df_parse = pd.read_csv("parse_context_final.csv")
df_street = pd.read_csv("street_normal.csv")
df_task_scramble = pd.read_csv("Scramble_FINAL.csv")
df_task_shuffle_words = pd.read_csv("Shuffling_wordings_FINAL.csv")

In [2]:
print(df_mcqn['input'][0] ," ***** ",df_mcqn['context'][0] ," ****** ",df_mcqn['target'][0])

O'Brien & Company , 811 First Ave Suite 380 ,  , US , WA , Seattle , 98104   *****   9830 overlook Dr  ,  ,  , US , Wa , Olympia , 98502  |  O'BRIEN & COMPANY , 811 1ST AVE STE 380 ,  , US , WA , SEATTLE , 98104-1434   |   O'Brien & Company , 811 First Avenue  Suite 380 ,  , US , WA , SEATTLE , 98104  ******   O'BRIEN & COMPANY , 811 1ST AVE STE 380 ,  , US , WA , SEATTLE , 98104-1434  |  O'Brien & Company , 811 First Avenue  Suite 380 ,  , US , WA , SEATTLE , 98104


# MAKING ALL DATASET TO UPPERCASE

In [3]:
df = df_mcqn.apply(lambda x: x.astype(str).str.upper())

In [4]:
df_mcqn = df

In [5]:
df_mcqs = df_mcqs.apply(lambda x: x.astype(str).str.upper())

In [6]:
df_parse = df_parse.apply(lambda x: x.astype(str).str.upper())

In [7]:
df_street = df_street.apply(lambda x: x.astype(str).str.upper())

In [8]:
df_task_scramble = df_task_scramble.apply(lambda x: x.astype(str).str.upper())

In [9]:
df_task_shuffle_words = df_task_shuffle_words.apply(lambda x: x.astype(str).str.upper())

In [10]:
df_task_shuffle_words

,target,shuffle_input
0,"111 YALE AVE N , APT212 , SEATTLE , WA , US","111 EYAL EVA N , PTA212 , LEETAST , AW"
1,"1121 HARRISON AVENUE , #115 , CENTRALIA , WA...","1121 OARSRHIN AVE # 115 , , RALNIACET , AW"
2,"5000 25TH AVE NE , APT 2207-C , SEATTLE , WA , US","5000 25TH AEV NE , TPA 2207-C , LTTSEEA , AW"
3,"11625 CENTER ROAD , UNIT D , EVERETT , WA , US","11625 NCRETE RD NITU D , , VEETRET , WA"
4,"1020 E JEFFERSON ST , 325 , SEATTLE , WA , US","1020 E FJEESRNOF TS 325 , , SETAETL , WA"
...,...,...
21949,"3025 LIMITED LN NW , STE 100 , OLYMPIA , WA , US","3025 ILTMEDI LN WN TES 100 , , OPAMIYL , WA"
21950,"17424 122ND AVE E , APT. B105 , PUYALLUP , WA ...","17424 122DN EVA E , TAP. B105 , LAUPYPLU , AW"
21951,"3013 99TH AVE NE , UNIT A , LAKE STEVENS , WA ...","3013 99TH AVE EN TNUI A , , LEAK NVSTEES , WA"
21952,"5240 UNIVERSITY WAY NE , APT607 , SEATTLE , WA...","5240 RINSIYETUV YWA EN , PTA607 , EASTTLE , AW"


In [11]:
df_mcqn.shape,df_mcqs.shape,df_street.shape,df_task_scramble.shape,df_task_shuffle_words.shape,df_parse.shape



((82614, 3), (33858, 3), (4542636, 2), (21954, 2), (21954, 2), (887195, 3))

In [12]:
df_mcqn.head(1)

,input,context,target
0,"O'BRIEN & COMPANY , 811 FIRST AVE SUITE 380 , ...","9830 OVERLOOK DR , , , US , WA , OLYMPIA ,...","O'BRIEN & COMPANY , 811 1ST AVE STE 380 , , ..."


In [13]:
df_mcqn['input'] = 'address : ' + df_mcqn['input'].astype(str)
df_mcqn['context'] = 'context : ' + df_mcqn['context'].astype(str)
df_mcqn["input_text"] = df_mcqn["input"].astype(str) +" $ " + df_mcqn["context"].astype(str)

In [14]:
df_mcqn['target'] = 'output : ' + df_mcqn['target'].astype(str)

In [15]:
df_mcqn.rename(columns = {'target':'target_text'}, inplace = True)

In [16]:
df_mcqn = df_mcqn.drop(['input','context'],axis=1)

In [17]:
df_mcqn['target_text'][0]

"output :  O'BRIEN & COMPANY , 811 1ST AVE STE 380 ,  , US , WA , SEATTLE , 98104-1434  |  O'BRIEN & COMPANY , 811 FIRST AVENUE  SUITE 380 ,  , US , WA , SEATTLE , 98104"

In [20]:
df_mcqn['input_text'][0]

"address : O'BRIEN & COMPANY , 811 FIRST AVE SUITE 380 ,  , US , WA , SEATTLE , 98104  $ context :  9830 OVERLOOK DR  ,  ,  , US , WA , OLYMPIA , 98502  |  O'BRIEN & COMPANY , 811 1ST AVE STE 380 ,  , US , WA , SEATTLE , 98104-1434   |   O'BRIEN & COMPANY , 811 FIRST AVENUE  SUITE 380 ,  , US , WA , SEATTLE , 98104"

In [21]:
df_mcqn

,target_text,input_text
0,"output : O'BRIEN & COMPANY , 811 1ST AVE STE ...","address : O'BRIEN & COMPANY , 811 FIRST AVE SU..."
1,"output : O'BRIEN & COMPANY , 811 FIRST AVE SUI...","address : O'BRIEN & COMPANY , 811 1ST AVE STE..."
2,"output : O'BRIEN & COMPANY , 811 FIRST AVE SUI...","address : O'BRIEN & COMPANY , 811 FIRST AVENU..."
3,"output : C/O MUD BAY , 711 78TH AVE SW , , ...","address : 711 78TH AVE SW , , , US , WA , TU..."
4,"output : 711 78TH AVE SW , , , US , WA , TUM...","address : C/O MUD BAY , 711 78TH AVE SW , ,..."
...,...,...
82609,"output : 101 6TH AVE S UNIT 313 , , , US , W...","address : 101 6TH AVE S UNIT 313 , , , US ,..."
82610,"output : 101 6TH AVE S UNIT 313 , , , US , W...","address : 101 6TH AVE S STREET UNIT 313 , , ..."
82611,"output : CROWD COW , 101 6TH AVE S APT 319 , ...","address : 101 6TH AVE S # 319 , , , US , WA ..."
82612,"output : 101 6TH AVE S # 319 , , , US , WA ,...","address : CROWD COW , 101 6TH AVE S APT 319 ,..."


In [22]:
df_mcqs['input'] = 'address : ' + df_mcqs['input'].astype(str)
df_mcqs['context'] = 'context : ' + df_mcqs['context'].astype(str)
df_mcqs["input_text"] = df_mcqs["input"].astype(str) +" $ " + df_mcqs["context"].astype(str)
df_mcqs['target_text'] = 'output : ' + df_mcqs['target'].astype(str)

In [26]:
df_mcqs= df_mcqs.drop(['input','context','target'],axis=1)

In [28]:
df_mcqs['input_text'][0]

'address : 424 100TH AVE SE ,  ,  , US , WA , OLYMPIA , 98501-9710  $ context :  608 W EMERSON ST APT 321 ,  ,  , US , WA , SEATTLE , 98119-1544   |   805 W EMERSON ST ,  ,  , US , WA , SEATTLE , 98119-1457  |  434 100TH AVE SE ,  ,  , US , WA , OLYMPIA , 98501 '

In [31]:
df_parse['prompt'] = 'address : ' + df_parse['prompt'].astype(str)
df_parse['context'] = 'context : ' + df_parse['context'].astype(str)
df_parse["input_text"] = df_parse["prompt"].astype(str) +" $ " + df_parse["context"].astype(str)
df_parse['target_text'] = 'output : ' + df_parse['response'].astype(str)

In [33]:
df_parse = df_parse.drop(['prompt','response','context'],axis=1)

In [37]:
df_parse['target_text'][0]

'output : STREET:18TH AVE;BUILDING:1133;UNIT:APT|4;CITY:SEATTLE;STATE:WA;POSTALCODE:98122 4701'

In [39]:
df_street
df_street['target_text'] = 'output : ' + df_street['target_text'].astype(str)
df_street['input'] = 'address : ' + df_street['input'].astype(str)
df_street.rename(columns = {'input':'input_text'}, inplace = True)

In [41]:
df_parse

,input_text,target_text
0,address : 1133 18TH AVE APT 4|||WA|SEATTLE|981...,output : STREET:18TH AVE;BUILDING:1133;UNIT:AP...
1,address : 1133 18TH AVE APT 4WASEATTLE98122-47...,output : STREET:18TH AVE;BUILDING:1133;UNIT:AP...
2,address : 1133 18TH AVE APT 4|||WA|98122-4701|...,output : STREET:18TH AVE;BUILDING:1133;UNIT:AP...
3,address : 1133 18TH AVE APT 498122-4701SEATTLE...,output : STREET:18TH AVE;BUILDING:1133;UNIT:AP...
4,address : !1133 18TH AVE APT 4!!WA!SEATTLE!981...,output : STREET:18TH AVE;BUILDING:1133;UNIT:AP...
...,...,...
887190,address : 12012 10TH PL NE||206|SEATTLE|WA|981...,output : STREET:NULL;BUILDING:NULL;UNIT:NULL;C...
887191,address : 4636 E MARGINAL WAY S STE 1206P NOAA...,output : STREET:NULL;BUILDING:NULL;UNIT:NULL;C...
887192,address : E 9028 DAY RDMEADWA99021 $ context :...,output : STREET:NULL;BUILDING:NULL;UNIT:NULL;C...
887193,address : |355 TRECK DR|PO# 2176252|TUKWILA|98...,output : STREET:NULL;BUILDING:NULL;UNIT:NULL;C...


In [43]:
df_task_scramble
df_task_scramble['target_text'] = 'output : ' + df_task_scramble['target'].astype(str)
df_task_scramble['input'] = 'address : ' + df_task_scramble['scramble_input'].astype(str)
df_task_scramble.rename(columns = {'scramble_input':'input_text'}, inplace = True)

In [45]:
df_task_scramble  = df_task_scramble.drop(['target','input'],axis=1)

In [47]:
df_task_scramble
df_task_scramble['input_text'] = 'address : ' + df_task_scramble['input_text'].astype(str)

In [53]:
df_task_scramble

,input_text,target_text
0,"address : YALE AVE , APT212 111 WA , N SEATTLE ,","output : 111 YALE AVE N , APT212 , SEATTLE , W..."
1,"address : CENTRALIA # AVE HARRISON , , 1121 11...","output : 1121 HARRISON AVENUE , #115 , CENTRA..."
2,"address : 2207-C APT AVE 25TH 5000 WA , NE SEA...","output : 5000 25TH AVE NE , APT 2207-C , SEATT..."
3,"address : , EVERETT , D RD UNIT WA , CENTER 11625","output : 11625 CENTER ROAD , UNIT D , EVERETT ..."
4,"address : , , JEFFERSON 1020 , 325 SEATTLE ST ...","output : 1020 E JEFFERSON ST , 325 , SEATTLE ,..."
...,...,...
21949,"address : OLYMPIA WA 100 , NW , LN STE 3025 , ...","output : 3025 LIMITED LN NW , STE 100 , OLYMPI..."
21950,"address : E 17424 , , PUYALLUP APT. , AVE WA 1...","output : 17424 122ND AVE E , APT. B105 , PUYAL..."
21951,"address : , STEVENS AVE LAKE UNIT 99TH NE , A ...","output : 3013 99TH AVE NE , UNIT A , LAKE STEV..."
21952,"address : NE APT607 UNIVERSITY , SEATTLE 5240 ...","output : 5240 UNIVERSITY WAY NE , APT607 , SEA..."


In [55]:
df_task_shuffle_words
df_task_shuffle_words['target_text'] = 'output : ' + df_task_shuffle_words['target'].astype(str)
df_task_shuffle_words['input_text'] = 'address : ' + df_task_shuffle_words['shuffle_input'].astype(str)
#df_task_shuffle_words.rename(columns = {'scramble_input':'input_text'}, inplace = True)

In [57]:
df_task_shuffle_words = df_task_shuffle_words.drop(['target','shuffle_input'],axis=1)

In [61]:
df_task_shuffle_words['input_text'][0]

'address : 111 EYAL EVA N , PTA212 , LEETAST , AW'

In [63]:
df_task_shuffle_words.head(5)

,target_text,input_text
0,"output : 111 YALE AVE N , APT212 , SEATTLE , W...","address : 111 EYAL EVA N , PTA212 , LEETAST , AW"
1,"output : 1121 HARRISON AVENUE , #115 , CENTRA...","address : 1121 OARSRHIN AVE # 115 , , RALNIAC..."
2,"output : 5000 25TH AVE NE , APT 2207-C , SEATT...","address : 5000 25TH AEV NE , TPA 2207-C , LTTS..."
3,"output : 11625 CENTER ROAD , UNIT D , EVERETT ...","address : 11625 NCRETE RD NITU D , , VEETRET ..."
4,"output : 1020 E JEFFERSON ST , 325 , SEATTLE ,...","address : 1020 E FJEESRNOF TS 325 , , SETAETL..."


In [66]:
df_task1 = df_mcqs
df_task2 = df_mcqn
df_task3 = df_parse
df_task6 = df_task_shuffle_words
df_task5 = df_task_scramble
df_task4 = df_street

In [67]:
df_task1['input_text'] = 'address_on_same_street: '+df_task1['input_text'].astype(str)
df_task2['input_text'] = 'address_on_same_neighbour: '+df_task2['input_text'].astype(str)
df_task3['input_text'] = 'parse : '+df_task3['input_text'].astype(str)
df_task4['input_text'] = 'street_normaliser : '+df_task4['input_text'].astype(str)
df_task5['input_text'] = 'correct address : '+df_task5['input_text'].astype(str)
df_task6['input_text'] = 'correct address : '+df_task6['input_text'].astype(str)


In [68]:
df_task1

,input_text,target_text
0,address_on_same_street: address : 424 100TH AV...,"output : 434 100TH AVE SE , , , US , WA , O..."
1,address_on_same_street: address : 434 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
2,address_on_same_street: address : 612 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
3,address_on_same_street: address : 612 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
4,address_on_same_street: address : 434 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
...,...,...
33853,address_on_same_street: address : 207 Z ST SE...,"output : 311 Z STREET , , , US , WA , TUMWAT..."
33854,address_on_same_street: address : 430 Z ST SE...,"output : 311 Z STREET , , , US , WA , TUMWAT..."
33855,address_on_same_street: address : 679 Z ST SE...,"output : 311 Z STREET , , , US , WA , TUMWAT..."
33856,address_on_same_street: address : 614 Z ST SE...,"output : 311 Z STREET , , , US , WA , TUMWAT..."


In [71]:
df_ans = pd.concat([df_task1,df_task2,df_task3,df_task4,df_task5,df_task6]).astype(str)


In [72]:
df_ans

,input_text,target_text
0,address_on_same_street: address : 424 100TH AV...,"output : 434 100TH AVE SE , , , US , WA , O..."
1,address_on_same_street: address : 434 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
2,address_on_same_street: address : 612 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
3,address_on_same_street: address : 612 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
4,address_on_same_street: address : 434 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
...,...,...
4542631,"correct address : address : 242 Z ST SE , , ...",output : Z_ST_SE
4542632,"correct address : address : 201 Z ST SE , , ...",output : Z_ST_SE
4542633,"correct address : address : 230 Z ST SE , , ...",output : Z_ST_SE
4542634,"correct address : address : 709 Z ST SE , , ...",output : Z_ST_SE


In [73]:
df_ans.to_csv('FINAL_v2.csv',index = False)

In [78]:
df_ans

,input_text,target_text
0,address_on_same_street: address : 424 100TH AV...,"output : 434 100TH AVE SE , , , US , WA , O..."
1,address_on_same_street: address : 434 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
2,address_on_same_street: address : 612 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
3,address_on_same_street: address : 612 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
4,address_on_same_street: address : 434 100TH A...,"output : 424 100TH AVE SE , , , US , WA , OL..."
...,...,...
4542631,"correct address : address : 242 Z ST SE , , ...",output : Z_ST_SE
4542632,"correct address : address : 201 Z ST SE , , ...",output : Z_ST_SE
4542633,"correct address : address : 230 Z ST SE , , ...",output : Z_ST_SE
4542634,"correct address : address : 709 Z ST SE , , ...",output : Z_ST_SE


# Training T5 Model

In [1]:
#!pip install datasets

In [83]:
import transformers
from datasets import load_dataset, load_metric
medium_datasets = load_dataset("csv", data_files="FINAL_v2.csv",cache_dir="~/SageMaker")

In [84]:
medium_datasets

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5590211
    })
})

In [85]:
datasets_train_test = medium_datasets["train"].train_test_split(test_size=5000)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=5000)
medium_datasets["train"] = datasets_train_validation["train"]
medium_datasets["validation"] = datasets_train_validation["test"]
medium_datasets["test"] = datasets_train_test["test"]

In [86]:
n_samples_train = len(medium_datasets["train"])
n_samples_validation = len(medium_datasets["validation"])
n_samples_test = len(medium_datasets["test"])
n_samples_total = n_samples_train + n_samples_validation + n_samples_test

print(f"- Training set: {n_samples_train*100/n_samples_total:.2f}%")
print(f"- Validation set: {n_samples_validation*100/n_samples_total:.2f}%")
print(f"- Test set: {n_samples_test*100/n_samples_total:.2f}%")

- Training set: 99.82%
- Validation set: 0.09%
- Test set: 0.09%


In [87]:
print(n_samples_train,n_samples_validation,n_samples_test,n_samples_total)

5580211 5000 5000 5590211


In [88]:
# making shufffling of dataset
medium_datasets["train"] = medium_datasets["train"].shuffle().select(range(n_samples_train))#131111260
medium_datasets["validation"] = medium_datasets["validation"].shuffle().select(range(n_samples_validation))
medium_datasets["test"] = medium_datasets["test"].shuffle().select(range(n_samples_test))
medium_datasets

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5580211
    })
    validation: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5000
    })
})

In [89]:
import warnings
warnings.filterwarnings('ignore')

In [90]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [91]:
#loading T5 base model
#The model was pre-trained on a on a multi-task mixture of unsupervised and supervised tasks
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [92]:
#prefix = "generate-scope-12-nak: "

max_input_length = 128
max_target_length = 256

# def clean_text(text):
#   sentences = nltk.sent_tokenize(text.strip())
#   sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
#   sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
#                                  if len(sent) > 0 and
#                                  sent[-1] in string.punctuation]
#   text_cleaned = "\n".join(sentences_cleaned_no_titles)
#   return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [text for text in examples["input_text"]]
  inputs = [text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["target_text"], max_length=max_target_length, 
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [93]:
medium_datasets

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5580211
    })
    validation: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 5000
    })
})

In [94]:
#medium_datasets_cleaned = medium_datasets.filter(lambda example: (len(example['text']) >= 500) and (len(example['title']) >= 20))
tokenized_datasets = medium_datasets.map(preprocess_data, batched=True)
tokenized_datasets

Map:   0%|          | 0/5580211 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5580211
    })
    validation: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [95]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [37]:
#!pip install transformers[torch]

In [38]:
#!pip install accelerate -U

In [96]:
batch_size = 30
model_name = "t5-base-task_v2"
model_dir = f"T5-Fine Tuning(ALL_GEO_DATA-Oct_v2)/Models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [97]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [41]:
#!pip install absl-py

In [42]:
#!pip install rouge_score

In [98]:
metric = load_metric("rouge")

In [99]:
import numpy as np
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #print("predictions.shape,labels.shape",predictions.shape, labels.shape)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    #print("decoded_preds.shape",len(decoded_preds))
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #print("decoded_labels.shape",len(decoded_labels))
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    # Compute F1 scores
    #print(decoded_preds[0][9:])
    #if(decoded_preds[0][9:] == 'UNIT' or )
    #print("f1_score : ",f1_score(decoded_labels, decoded_preds, average='weighted'))
    # Comput confusion matrics
    #cm = confusion_matrix(decoded_labels, decoded_preds)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    #disp.plot()
    #plt.show()

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    #print(result)
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [45]:
#!pip install tensorboardX

In [100]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [101]:
tokenized_datasets["train"].set_format(type='pandas')
df = tokenized_datasets["train"][:]

In [102]:
# Start TensorBoard before training to monitor it in progress
%load_ext tensorboardX
%tensorboardX --logdir '{model_dir}'/runs

The tensorboardX module is not an IPython extension.


UsageError: Line magic function `%tensorboardX` not found.


In [103]:
import torch
torch.cuda.empty_cache()

#### Training T5

In [104]:
# New One
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,1.807000,0.360348,86.872500,80.177300,86.701000,86.689400,11.950400
200,0.367500,0.203876,89.393000,83.364000,89.271500,89.276900,11.877600
300,0.240900,0.161503,90.057500,84.443000,89.944800,89.944400,11.867000
400,0.195100,0.142437,90.361200,85.290800,90.263500,90.266900,11.880200
500,0.171800,0.128433,90.495500,85.930700,90.405300,90.424500,11.956200
600,0.155600,0.119837,90.734500,86.540800,90.639800,90.658300,11.999800
700,0.147400,0.111088,90.783900,86.664800,90.701400,90.720800,12.011600
800,0.134800,0.107238,90.874600,86.835400,90.806000,90.818500,12.006200
900,0.131900,0.103262,90.940000,86.999200,90.869000,90.883900,12.021600
1000,0.126900,0.099525,90.954600,86.940600,90.890300,90.902300,12.004400


TrainOutput(global_step=23251, training_loss=0.07315952948798754, metrics={'train_runtime': 20746.0485, 'train_samples_per_second': 268.977, 'train_steps_per_second': 1.121, 'total_flos': 1.8864927015488717e+17, 'train_loss': 0.07315952948798754, 'epoch': 1.0})

In [105]:
trainer.save_model()

In [13]:
# model_name = "t5-large-scope_12_nak-generation"
# model_dir = f"T5-Fine Tuning(GEO_DATA-6th Sept)/Models/{model_name}"
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer
model_name = "t5-base-task_more_epochs"
model_dir = f"T5-Fine Tuning(ALL_GEO_DATA-After 22nd Sept_more_epochs)/Models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 128

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
text = 'address : line1 : 1617 YALE AVE, line2 : APARTMENT B-205,line3 : ,state : SEATLE, city : WA,pincode : 98101-1900'

inputs = ["Parse address : " + text]
print(inputs)
inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=256)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
#print(decoded_output)
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
print(predicted_title)